In [0]:
import numpy as np
from sympy.integrals.quadrature import gauss_lobatto

!pip install numdifftools
import numdifftools as nd

# Metodo di Newton-Raphson

La funzione **newton** prende in input un array $X$, generato con *dataset_full*, ed il relativo valore della dimensione $d$.

I primi $d$ valori di $X$ sono le coordinate del punto $x$ di cui vogliamo calcolare la preimmagine, mentre i restanti $d \cdot n^d$ sono le coordinate degli $n^d$ punti di supporto di $K$.

Ci restituisce in output un array contenente le $d$ coordinate della preimmagine di $x$, ovvero le coordinate di $\hat{x}: \mathcal{L}_K(\hat{x}) = x$. 

Settando il parametro *eps* (default $10^{-8}$) si può impostare la precisione richiesta al metodo.

In [0]:
def newton(X, d, eps=1.e-8):
    
    p = X[0:d]
    K = X[d:].reshape(-1,d)
    n = int(pow(K.shape[0],1/d)+.5)

    ##################################################################

    q, w = gauss_lobatto(n, 10)
    q = np.array(q, dtype=float)
    if(d==2):
        Q1, Q2 = np.meshgrid(q,q)
        Khat = np.stack([Q1,Q2], axis=-1).reshape((-1,d))
    elif(d==3):
        Q1, Q2, Q3 = np.meshgrid(q,q,q)
        Khat= np.stack([Q1,Q2,Q3], axis=-1).reshape((-1,d))

    ##################################################################
    
    def lagrange(i,x):
        x = x.repeat(n-1)
        v = []
        for k in range(d):
            v.append(np.delete(q,np.where(q==Khat[i][k])))
        v = np.array(v).flatten()
        avoid = Khat[i].repeat(n-1)
        return np.prod((x-v)/(avoid-v))
    
    def L(x):
        lgr = np.array([lagrange(i,x) for i in range(pow(n,d))])
        return np.matmul(lgr,K)
    
    ##################################################################
    
    xhat = np.zeros(d)
    f = lambda xhat: L(xhat) - p
    while(np.linalg.norm(f(xhat))>eps):
          xhat = xhat - np.matmul(np.linalg.inv(nd.Jacobian(f)(xhat)), f(xhat).T)
    
    return xhat